In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import bs4 as bs
import requests
import bs4 as bs
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import re
folderpath=r"C:\Users\Zoe\Desktop\maz-datenjournalismus\Daten_BR_Handelsreg"

# Gemeinden und Stadt zusammenbringen
Da ich für die Stadt bereits alles bereinigt hatte muss, muss ich dies nun auch für die Gemeinden machen. Das Gemeindeframe und das Stadtframe müssen dann zusammenbebracht werden.


In [2]:
#Gemeinden
df_adan=pd.read_csv(folderpath+"/BRFrames/Alle_Gemeinden_ausgelesen_roh")

## Zahl und Adresse rausziehen 
evtl. könnte man die Frames als erstes wieder zusammenfügen
zuerst alle aussortieren, die überhaupt eine Zahl enthalten
Danach splitten oder mit Regex. Da es nur eine Zahl gibt, wäre hier vielleicht Regex besser.


In [3]:
df_adan

,Unnamed: 0,Seite,Anzahl,Adresse
0,0,frontpage0.htm,\n\t\t\t\tNo. of companies found: 1\n\t\t\t\t,"<input aria-disabled=""false"" aria-multiline=""f..."
1,1,frontpage0_0.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
2,2,frontpage0_1.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
3,3,frontpage0_10.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
4,4,frontpage0_100.htm,0,"<input aria-disabled=""false"" aria-multiline=""f..."
...,...,...,...,...
16108,16108,frontpage9_2035.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
16109,16109,frontpage9_2036.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
16110,16110,frontpage9_2037.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
16111,16111,frontpage9_2038.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."


In [4]:
df_adan[df_adan["Anzahl"].str.contains("SearchReset")==True]

,Unnamed: 0,Seite,Anzahl,Adresse
1,1,frontpage0_0.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
1013,1013,frontpage10_2040.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
1014,1014,frontpage10_2041.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
1015,1015,frontpage10_2042.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
1016,1016,frontpage10_2043.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
...,...,...,...,...
16107,16107,frontpage9_2034.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
16108,16108,frontpage9_2035.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
16109,16109,frontpage9_2036.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."
16110,16110,frontpage9_2037.htm,SearchReset search criteria,"<input id=""idSucheForm:panel:idDiverseTyp_focu..."


In [ ]:
df_adan.shape #Ausganglage waren 4690. Offenbar habe ich einige doppelt. egal.

### Zahl

In [5]:
# Hier nun die Formel, um aus allen die Zahl rausziehen zu können
df_adan["Anzahl"]=df_adan["Anzahl"].str.extract("(\d+)")

### Adresse

In [6]:
df_adan["Adresse"]=df_adan["Adresse"].str.extract("value=\"(.*)\"")

In [8]:
df_adan[df_adan["Anzahl"].notnull()==False]

,Unnamed: 0,Seite,Anzahl,Adresse
1,1,frontpage0_0.htm,NaN,NaN
1013,1013,frontpage10_2040.htm,NaN,NaN
1014,1014,frontpage10_2041.htm,NaN,NaN
1015,1015,frontpage10_2042.htm,NaN,NaN
1016,1016,frontpage10_2043.htm,NaN,NaN
...,...,...,...,...
16107,16107,frontpage9_2034.htm,NaN,NaN
16108,16108,frontpage9_2035.htm,NaN,NaN
16109,16109,frontpage9_2036.htm,NaN,NaN
16110,16110,frontpage9_2037.htm,NaN,NaN


In [9]:
# Offenbar hat es bei einigen Seiten nichts eingegeben. Nun muss ich mit den Andressen abgleichen, welche SEiten das sind.
df_gem_adr=pd.read_csv(folderpath+"/BRFrames/Alle_Adressen_ohne_Stadt")

In [10]:
df_abgleich=pd.merge(df_adan, df_gem_adr, on="Adresse", how="outer") 

In [11]:
df_abgleich

,Unnamed: 0_x,Seite,Anzahl,Adresse,Unnamed: 0_y,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP
0,0.0,frontpage0.htm,1,"Marktgasse 12, Baar",0.0,310003.0,0.0,ZG,1701.0,Baar,Marktgasse,12,6340.0,0.0,Baar,2682511.588,1227773.739,de
1,9000.0,frontpage20_0.htm,1,"Marktgasse 12, Baar",0.0,310003.0,0.0,ZG,1701.0,Baar,Marktgasse,12,6340.0,0.0,Baar,2682511.588,1227773.739,de
2,1.0,frontpage0_0.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1013.0,frontpage10_2040.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1014.0,frontpage10_2041.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24103,NaN,NaN,NaN,"Hinterbalis, Walchwil",789.0,191581692.0,0.0,ZG,1710.0,Walchwil,Hinterbalis,NaN,6318.0,0.0,Walchwil,2682918.805,1218133.202,de
24104,NaN,NaN,NaN,"Sattel, Walchwil",818.0,191696853.0,0.0,ZG,1710.0,Walchwil,Sattel,NaN,6318.0,0.0,Walchwil,2682704.700,1219514.850,de
24105,NaN,NaN,NaN,"Freudenberg, Walchwil",850.0,191730019.0,0.0,ZG,1710.0,Walchwil,Freudenberg,NaN,6318.0,0.0,Walchwil,2681806.510,1218036.750,de
24106,NaN,NaN,NaN,"Mittelschwändi, Walchwil",873.0,191730101.0,0.0,ZG,1710.0,Walchwil,Mittelschwändi,NaN,6318.0,0.0,Walchwil,2682263.160,1218945.140,de


In [13]:
df_not_anz=df_abgleich[df_abgleich["Anzahl"].notnull()==False]

In [15]:
df_not_anz.shape

(10829, 18)

In [16]:
df_fehlende_gmd=df_not_anz[df_not_anz["Seite"].notnull()==False]

In [18]:
df_fehlende_gmd.drop_duplicates("Adresse")

,Unnamed: 0_x,Seite,Anzahl,Adresse,Unnamed: 0_y,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP
16113,NaN,NaN,NaN,"Unterbrüglenweg 3, Baar",443.0,310531.0,0.0,ZG,1701.0,Baar,Unterbrüglenweg,3,6340.0,0.0,Baar,2681623.439,1228361.557,de
16114,NaN,NaN,NaN,"Unterbrüglenweg 5, Baar",444.0,310532.0,0.0,ZG,1701.0,Baar,Unterbrüglenweg,5,6340.0,0.0,Baar,2681608.183,1228354.331,de
16115,NaN,NaN,NaN,"Unterbrüglenweg 7, Baar",445.0,310533.0,0.0,ZG,1701.0,Baar,Unterbrüglenweg,7,6340.0,0.0,Baar,2681589.222,1228344.400,de
16116,NaN,NaN,NaN,"Unterbrüglenweg 2, Baar",446.0,310534.0,0.0,ZG,1701.0,Baar,Unterbrüglenweg,2,6340.0,0.0,Baar,2681633.368,1228389.241,de
16117,NaN,NaN,NaN,"Unterbrüglenweg 4, Baar",447.0,310535.0,0.0,ZG,1701.0,Baar,Unterbrüglenweg,4,6340.0,0.0,Baar,2681608.947,1228385.977,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24103,NaN,NaN,NaN,"Hinterbalis, Walchwil",789.0,191581692.0,0.0,ZG,1710.0,Walchwil,Hinterbalis,NaN,6318.0,0.0,Walchwil,2682918.805,1218133.202,de
24104,NaN,NaN,NaN,"Sattel, Walchwil",818.0,191696853.0,0.0,ZG,1710.0,Walchwil,Sattel,NaN,6318.0,0.0,Walchwil,2682704.700,1219514.850,de
24105,NaN,NaN,NaN,"Freudenberg, Walchwil",850.0,191730019.0,0.0,ZG,1710.0,Walchwil,Freudenberg,NaN,6318.0,0.0,Walchwil,2681806.510,1218036.750,de
24106,NaN,NaN,NaN,"Mittelschwändi, Walchwil",873.0,191730101.0,0.0,ZG,1710.0,Walchwil,Mittelschwändi,NaN,6318.0,0.0,Walchwil,2682263.160,1218945.140,de


In [20]:
df_fehlende_gmd.to_csv(folderpath+"/BRFrames/fehlende_Gemeinden_nicht_gelesen")

In [23]:
df_fehlende_halb1=df_fehlende_gmd.iloc[:4000,:]#Frame halbieren um auf zwei PCs laufen zu lassen

In [24]:
df_fehlende_halb2=df_fehlende_gmd.iloc[4000:,:]

In [25]:
df_fehlende_halb1.to_csv(folderpath+"/BRFrames/fehlende_Gemeinden_Teil1")

In [26]:
df_fehlende_halb2.to_csv(folderpath+"/BRFrames/fehlende_Gemeinden_Teil2")